## DenseNetwork 구현 및 학습  
resNet을 잘계승하고 있는 Modern한 알고리즘 입니다.  
그렇다고 무조건 denseNet이 좋은것은 하닙니다. 어떤것은 resNet이 어떤것은 denseNet이 적합합니다.  
실제로도 resNet은 학습이 안정적으로 적용되는 반면 densNet은 값이 흔들립니다.

In [7]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [8]:
EPOCHS = 10

## Dense Unit 구현

In [16]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn=tf.keras.layers.BatchNormalization()
        self.conv=tf.keras.layers.Conv2D(filter_out,kernel_size,padding='same')
        self.concat=tf.keras.layers.Concatenate()

    def call(self, x, training=False, mask=None): #x:(Batch,H,W,Ch_in)
        h=self.bn(x,training=training)
        h=tf.nn.relu(h)
        h=self.conv(h) # h: (Batch,H,W,filter_ouutput)
        return self.concat([x,h])#(Batch,H,W,Ch_in+filter_output))

## DenseLayer 구현  
여기서 growth_rate는 dense unit이 한번 콜이 될 때마다 input 채널에서 output 필터 개수만큼 점점 커지기 때문에 커지는 특성을 growth rate라고 표현한다.

In [17]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence=list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate,kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x=unit(x, training=training)
        return x

## Transition Layer 구현

In [18]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.pool=tf.keras.layers.MaxPool2D()

    def call(self, x, training=False, mask=None):
        x=self.conv(x)
        return self.pool(x)

## 모델 정의

In [19]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')
        
        self.dl1=DenseLayer(2,4,(3,3))#28x28x16
        self.tr1=TransitionLayer(16,(3,3))#14x14x16
        
        self.dl2=DenseLayer(2,8,(3,3))#14x14x16
        self.tr2=TransitionLayer(32,(3,3))#7x7x32
        
        self.dl3=DenseLayer(2,16,(3,3))#7x7x64        
        
        self.flatten=tf.keras.layers.Flatten()
        self.dense1=tf.keras.layers.Dense(128,activation='relu')
        self.dense2=tf.keras.layers.Dense(10,activation='softmax')
        
        
    def call(self, x, training=False, mask=None):
        x=self.conv1(x)
        
        x=self.dl1(x,training=training)
        x=self.tr1(x)
        
        x=self.dl2(x,training=training)
        x=self.tr2(x)
        
        x=self.dl3(x)
        
        x= self.flatten(x)
        x=self.dense1(x)
        return self.dense2(x)
        

## 학습, 테스트 루프 정의

In [20]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images,training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images,training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [21]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [22]:
# Create model
model = DenseNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [23]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.11651317775249481, Accuracy: 96.63833618164062, Test Loss: 0.12777933478355408, Test Accuracy: 96.5
Epoch 2, Loss: 0.05911349505186081, Accuracy: 98.2933349609375, Test Loss: 0.08135871589183807, Test Accuracy: 98.00999450683594
Epoch 3, Loss: 0.046942953020334244, Accuracy: 98.7249984741211, Test Loss: 0.05912934988737106, Test Accuracy: 98.25999450683594
Epoch 4, Loss: 0.04083023592829704, Accuracy: 98.875, Test Loss: 0.06964223831892014, Test Accuracy: 98.22000122070312
Epoch 5, Loss: 0.03901837393641472, Accuracy: 98.9383316040039, Test Loss: 0.056346096098423004, Test Accuracy: 98.7699966430664
Epoch 6, Loss: 0.03419676050543785, Accuracy: 99.1199951171875, Test Loss: 0.0459163524210453, Test Accuracy: 98.8699951171875
Epoch 7, Loss: 0.025894753634929657, Accuracy: 99.31832885742188, Test Loss: 0.06836067885160446, Test Accuracy: 98.44999694824219
Epoch 8, Loss: 0.029866192489862442, Accuracy: 99.24166870117188, Test Loss: 0.058431558310985565, Test Accuracy: 98.8